<a href="https://colab.research.google.com/github/ahillahaffat/pca_raw/blob/master/UTS_Pengenalan_Pola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdallahalidev/plantvillage-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/abdallahalidev/plantvillage-dataset/versions/3


In [ ]:
def mean_centering(x):
    jumlah_total = 0
    jumlah_elemen = len(x) * len(x[0])
    for i in range(len(x)):
        for j in range(len(x[0])):
            jumlah_total += x[i][j]
    rata_rata = jumlah_total / jumlah_elemen
    x_terpusat = [[x[i][j] - rata_rata for j in range(len(x[0]))] for i in range(len(x))]
    return x_terpusat

x = [
    [3.5, 0.5, 4.5],
    [0.5, 3.0, 1.0],
    [4.5, 0.0, 3.5],
    [1.0, 2.0, 0.0]
]
x_terpusat = mean_centering(x)

print("Dataset yang sudah di Mean Centering:\n", x_terpusat)

Dataset yang sudah di Mean Centering:
 [[1.5, -1.5, 2.5], [-1.5, 1.0, -1.0], [2.5, -2.0, 1.5], [-1.0, 0.0, -2.0]]


**Menghitung Matriks Kovarians (21081010176 - Annisa Rachmania)**

In [ ]:
# uji coba
x_terpusat = [
     [1.5, -0.5, 2.0],
     [-1.0, 1.5, -0.5],
     [2.0, -1.5, 0.5],
     [-1.5, 1.0, -2.0]
]

In [ ]:
def hitung_matriks_kovarian(x_terpusat):
    jumlah_sampel = len(x_terpusat)
    jumlah_fitur = len(x_terpusat[0])
    matriks_kovarian = [[0 for _ in range(jumlah_fitur)] for _ in range(jumlah_fitur)]

    # Hitung rata-rata tiap fitur
    means = [sum(x[i] for x in x_terpusat) / jumlah_sampel for i in range(jumlah_fitur)]

    # Hitung matriks kovarian
    for i in range(jumlah_fitur):
        for j in range(jumlah_fitur):
            total = 0
            for k in range(jumlah_sampel):
                total += (x_terpusat[k][i] - means[i]) * (x_terpusat[k][j] - means[j])
            matriks_kovarian[i][j] = total / (jumlah_sampel - 1)

    return matriks_kovarian

In [ ]:
matriks_kovarian = hitung_matriks_kovarian(x_terpusat)

In [ ]:
print(matriks_kovarian)

[[3.0833333333333335, -2.2916666666666665, 2.5], [-2.2916666666666665, 1.8958333333333333, -1.5], [2.5, -1.5, 2.8333333333333335]]


## **Eigen Values**

In [ ]:
def hitung_eigenvalues(matriks):
    n = len(matriks)
    toleransi = 1e-10
    max_iter = 100

    A = [row[:] for row in matriks]
    eigenvals = []

    for i in range(n):
        val = A[i][i]
        iter_count = 0

        while iter_count < max_iter:
            delta = sum(abs(A[i][j]) for j in range(n) if i != j)

            if delta < toleransi:
                break

            for j in range(n):
                if j != i:
                    if abs(A[i][j]) > toleransi:
                        r = ((A[i][i]**2 + A[i][j]**2)**0.5)
                        c = A[i][i]/r
                        s = -A[i][j]/r

                        temp_i = A[i][:]
                        temp_j = A[j][:]
                        for k in range(n):
                            A[i][k] = c*temp_i[k] - s*temp_j[k]
                            A[j][k] = s*temp_i[k] + c*temp_j[k]

                        for k in range(n):
                            temp_i = A[k][i]
                            temp_j = A[k][j]
                            A[k][i] = c*temp_i - s*temp_j
                            A[k][j] = s*temp_i + c*temp_j

            val = A[i][i]
            iter_count += 1

        eigenvals.append(val)

    return eigenvals

# **Eigen Vector**

In [ ]:
def hitung_eigenvectors(matriks, eigenvalues):
    n = len(matriks)
    eigenvectors = []
    toleransi = 1e-10

    for eigenval in eigenvalues:
        temp_matrix = [[matriks[i][j] - (eigenval if i == j else 0)
                       for j in range(n)] for i in range(n)]

        vector = [0] * n
        vector[0] = 1

        for i in range(n-1, -1, -1):
            sum_val = sum(temp_matrix[i][j] * vector[j] for j in range(n))
            if abs(temp_matrix[i][i]) > toleransi:
                vector[i] = -sum_val / temp_matrix[i][i]

        magnitude = sum(x*x for x in vector) ** 0.5
        if magnitude > toleransi:
            vector = [x/magnitude for x in vector]

        eigenvectors.append(vector)

    return eigenvectors

#  Result From


*   Eigen Vectors
*   Eigen Values



In [ ]:
print("\nEigenvalues:")
for i, val in enumerate(eigenvalues):
    print(f"λ{i+1} = {val:.4f}")

print("\nEigenvectors:")
for i, vec in enumerate(eigenvectors):
    print(f"v{i+1} = {[f'{x:.4f}' for x in vec]}")


Eigenvalues:
λ1 = 6.9178
λ2 = 0.8810
λ3 = 0.0137

Eigenvectors:
v1 = ['-0.2402', '-0.7011', '0.6714']
v2 = ['0.5307', '0.2326', '-0.8150']
v3 = ['0.1007', '0.4968', '-0.8620']


In [ ]:
# Data eigenvalues dan eigenvectors dari hasil sebelumnya
eigenvalues = [6.9178, 0.8810, 0.0137]
eigenvectors = [
    [-0.2402, -0.7011, 0.6714],
    [0.5307, 0.2326, -0.8150],
    [0.1007, 0.4968, -0.8620]
]

def urutkan_eigen(eigenvalues, eigenvectors):
        pairs = list(zip(eigenvalues, eigenvectors))
        pairs.sort(reverse=True, key=lambda x: x[0])

        sorted_values = [pair[0] for pair in pairs]
        sorted_vectors = [pair[1] for pair in pairs]

        return sorted_values, sorted_vectors


In [ ]:
# Testing
sorted_values, sorted_vectors = urutkan_eigen(eigenvalues, eigenvectors)
print("Eigenvalues terurut:")
for i, val in enumerate(sorted_values):
    print(f"λ{i+1} = {val:.4f}")

print("\nEigenvectors terurut:")
for i, vec in enumerate(sorted_vectors):
    print(f"v{i+1} = {[f'{x:.4f}' for x in vec]}")

Eigenvalues terurut:
λ1 = 6.9178
λ2 = 0.8810
λ3 = 0.0137

Eigenvectors terurut:
v1 = ['-0.2402', '-0.7011', '0.6714']
v2 = ['0.5307', '0.2326', '-0.8150']
v3 = ['0.1007', '0.4968', '-0.8620']


# KOMPONEN UTAMANYA

In [ ]:
def pilih_komponen_utama(sorted_values, sorted_vectors, n_components):
    selected_values = sorted_values[:n_components]
    selected_vectors = sorted_vectors[:n_components]

    return selected_values, selected_vectors

n_components = 2
selected_values, selected_vectors = pilih_komponen_utama(sorted_values, sorted_vectors, n_components)

print(f"\nHasil {n_components} komponen utama:")
print("Eigenvalues terpilih:")
for i, val in enumerate(selected_values):
    print(f"λ{i+1} = {val:.4f}")


Hasil 2 komponen utama:
Eigenvalues terpilih:
λ1 = 6.9178
λ2 = 0.8810


In [ ]:
def hitung_proporsi_variansi(eigenvalues):
    total = sum(eigenvalues)
    proportions = [val/total * 100 for val in eigenvalues]

    cumulative = []
    current_sum = 0
    for prop in proportions:
        current_sum += prop
        cumulative.append(current_sum)

    return proportions, cumulative

proportions, cumulative = hitung_proporsi_variansi(sorted_values)

print("\nProporsi variansi yang dijelaskan:")
for i, (prop, cum) in enumerate(zip(proportions, cumulative)):
    print(f"Komponen {i+1}: {prop:.2f}% (Kumulatif: {cum:.2f}%)")


Proporsi variansi yang dijelaskan:
Komponen 1: 88.55% (Kumulatif: 88.55%)
Komponen 2: 11.28% (Kumulatif: 99.82%)
Komponen 3: 0.18% (Kumulatif: 100.00%)


**Membentuk Matriks Proyeksi (Ananda Asa Firstha)**

In [ ]:
# Ujicoba Vektor eigen
eigenvectors = [
    [0.5, 0.5, -0.7],
    [-0.7, 0.3, 0.5],
    [0.5, -0.8, 0.3]
]

# Jumlah komponen utama yang ingin dipilih
num_components = 2

def bentuk_matriks_proyeksi(vektor_eigen, jumlah_komponen):

    # Pilih jumlah_komponen teratas dari setiap vektor eigen
    matriks_proyeksi = [baris[:jumlah_komponen] for baris in vektor_eigen]
    return matriks_proyeksi

# Panggil fungsi untuk membentuk matriks proyeksi
matriks_proyeksi = bentuk_matriks_proyeksi(eigenvectors, num_components)

# Cetak hasil matriks proyeksi
print("Matriks Proyeksi:")
for baris in matriks_proyeksi:
    print(baris)


Matriks Proyeksi:
[0.5, 0.5]
[-0.7, 0.3]
[0.5, -0.8]


**Transformasi data ke ruang komponen utama (21081010283 - Dzaru Rizky Fathan Fortuna)**

---



In [ ]:
# Ini uji percobaan fungsi ya ges
# x_centered = [
#     [1.5, -0.5, 2.0],
#     [-1.0, 1.5, -0.5],
#     [2.0, -1.5, 0.5],
#     [-1.5, 1.0, -2.0]
# ]

# eigenvectors = [
#     [0.5, 0.5, -0.7],
#     [-0.7, 0.3, 0.5],
#     [0.5, -0.8, 0.3]
# ]

# num_components = 2

In [ ]:
def proyeksi_data(x_terpusat, vektor_eigen, jumlah_komponen) :
  x_tereduksi = []
  for baris in x_terpusat :
    baris_proyeksi = []
    for i in range(jumlah_komponen) :
      proyeksi = 0
      for j in range(len(baris)) :
        proyeksi += baris[j] * vektor_eigen[i][j]
      baris_proyeksi.append(proyeksi)
    x_tereduksi.append(baris_proyeksi)
  return x_tereduksi

In [ ]:
# proyeksi_data(x_centered, eigenvectors, num_components)